## Libraries

In [1]:
import tempfile
import math
import os
from subprocess import call
import matplotlib.pyplot as plt
from f90nml import *
from array import *
import numpy as np
import pandas as pd 
import seaborn as sns
from statistics import *
import shutil
import pdb
import sys
import time
from numpy import mean
from numpy import std
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import linregress
from datetime import datetime


#read in printFolder, makeDefNameList, newFolder, deleteFolder, readOutput from coreFunctions.py
from coreFunctions import *

#read in plotting functionality from plottingFunctions.py
from plottingFunctions import *

call("rm -rf tmp*",shell=True)

dfModel = []#will contain output from our model

## Run Model

In [2]:
start = time.time()
nameList = makeDefNamelist() #make namelist
#-------------------------------------Boolean Inputs--------------------------------------------------
plot=False;#make the fancy plots?
save=False#save the plots
coupled=False; #if this is False, program quits once EBM reaches equilibrium
analyze = False; #whether or not to analyze the population parts of model
lverbose=False; #controls terminal output, used for debugging
showInputs = False; #prints the namelist before running the program
#-------------------------------------Other Important Inputs-------------------------------------------------# maxPopList = [10000,40000,70000,100000] #specify all maxpops to use
exp =0
runTime=4000                          #Change runtime           (years)
#----------------------------------Set NameList Values to Specified Inputs---------------------
verbose=False
carryCap=10000
#------------------------------------Run the Model----------------------------------------------------
n = 40
nameList['ebm']['lverbose'] = verbose
distanceList = [] #x
tempList = [] #y
dTdPList = []
dimVarList = []
pco2List = []
popDeathList = [] #amount of people dead within 2 generations of the peak
declineTime = [] #time for population to decline by 10% of its peak (any numbers over 500 are approximations)
anthroPopList = [] #anthrprogenic population
tempErrorList = [] #gives the absolute value of the difference between the goal equilibrium temp and that achieved by the returned value of pco2
peakPopList  = [] #max population reached
carryCapList = []
distances = np.linspace(0.934, 1.147, n)
goalEqTemps = np.linspace(273.15,373.15, n)

numRuns = len(distances)*len(goalEqTemps)
counter = 0

for dist in distances:
    end = time.time()
    timeMinutes = (end - start)/60
    timeHours = timeMinutes/60
    print(f"\n Runtime: {timeMinutes:.2f} minutes have passed")
    print(f" Runtime: {timeHours:.2f} hours have passed\n ")
    print("\n", "Distance: ", round(dist,3), " AU")
    for temp in goalEqTemps:
        carryCapList.append(carryCap/1000)
        distanceList.append(dist)
        tempList.append(temp)
        goalPco2, errorOnTemp = pco2Finder(temp, nameList, [dist], [carryCap], lverbose=verbose)#False = suppress output
        eq = False
        tempErrorList.append(errorOnTemp)
        if np.isnan(goalPco2): popDeathList.append(np.nan)
        else:
            nameList['ebm']['pco20'] = goalPco2*10**-6
            dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, True, 3000, False, False, False,"driver.exe",[carryCap],[dist],showInputs, analyzeVerbose=False, experiment=0, scaleInitPop=True)
            popDeathList.append(popDeath[0])
        pco2List.append(goalPco2)
        goaldTdP = dTdPFinder(goalPco2, dist, nameList, [carryCap])
        dTdPList.append(goaldTdP)
        anthroPop = (nameList['ebm']['rBirth0']*nameList['ebm']['dtemp'])/(nameList['ebm']['rco2']*goaldTdP)
        anthroPopList.append(anthroPop/1000)
        if eq:
            popStats = analyzeRun(dfModel,nameList, anthroPop, verbose)
            print(f"Distance: {dist:.1f}, Temp: {temp:.1f}, MaxPop: {popStats['maxPop']:.0f}")
            peakPopList.append(popStats['maxPop'])
            declineTime.append(popStats['tDecline'])
        else:
            declineTime.append(np.nan)
            peakPopList.append(np.nan)
#        print(f"dTdP: {goaldTdP:.3e}")
        if not np.isnan(goalPco2): dimVar=(carryCap*nameList['ebm']['rco2']*goaldTdP)/(nameList['ebm']['rBirth0']*nameList['ebm']['dtemp'])
        if np.isnan(goalPco2): dimVar = np.nan
        dimVarList.append(dimVar)
        print(str(round((counter/numRuns)*100,2)) + "% Completion" )
        counter += 1
#        print(f"Dimensionless Variable: {dimVar}","\n")
#-------------------------------------------------------
df = pd.DataFrame()
df["dist"] = distanceList #x
df["temp"] = tempList     #y
df["dTdP"] = dTdPList
df["pco20"] = pco2List
df["gamma"] = dimVarList
df["popDeath"] = popDeathList
df["declineTime"] = declineTime
df["peakPopList"] = peakPopList
df["carryCapList"] = carryCapList
df["anthroPopList"] = anthroPopList
df["tempError"] = tempErrorList
#---------------------------------------------------------
df["logGamma"] = np.log10(df.gamma)
df["logdTdP"] = np.log10(df.dTdP)
df["logpco20"] = np.log10(df.pco20)
df["logpopDeath"] = np.log10(df.popDeath)
df["logdeclineTime"] = np.log10(df.declineTime)
df["logAnthroPop"] = np.log10(df.anthroPopList)
#----------------------------------------------------------
now = datetime.now() # current date and time
month = now.strftime("%m")
day = now.strftime("%d")
dateToday = f"{month}_{day}"
df.to_csv("dataSweep_" + dateToday +"_NMax_10_Billion.csv")
#--------------------------------------------------------
# A=0.04;#birth ratee
# tFreeFall = lambda g: min(1, 1/g)
# df["tFreeFall"] = df["gamma"].apply(tFreeFall)
# df["tGen"] = df["tFreeFall"]/A
#-----------------------------------------------------
end = time.time()
timeSeconds = end-start
timeMinutes = (timeSeconds)/60
timeHours = timeMinutes/60
print(f"Runtime: {timeMinutes:.2f} minutes")
print(f"Runtime: {timeHours:.2f} hours")
print(f"Average Time to advance 1%: {timeSeconds/100} s")
call("echo End of Python Notebook Reached", shell=True);


 Runtime: 0.00 minutes have passed
 Runtime: 0.00 hours have passed
 

 Distance:  0.934  AU
0.0% Completion
0.06% Completion
0.12% Completion
0.19% Completion
0.25% Completion
0.31% Completion
0.38% Completion
0.44% Completion
0.5% Completion
0.56% Completion
0.62% Completion
0.69% Completion
0.75% Completion
0.81% Completion
0.88% Completion
0.94% Completion
1.0% Completion
1.06% Completion
1.12% Completion
1.19% Completion
1.25% Completion
1.31% Completion
1.38% Completion
Distance: 0.9, Temp: 332.1, MaxPop: 0
1.44% Completion
Distance: 0.9, Temp: 334.7, MaxPop: 0
1.5% Completion
Distance: 0.9, Temp: 337.3, MaxPop: 0
1.56% Completion
Distance: 0.9, Temp: 339.8, MaxPop: 0
1.62% Completion
Distance: 0.9, Temp: 342.4, MaxPop: 0
1.69% Completion
Distance: 0.9, Temp: 344.9, MaxPop: 0
1.75% Completion
Distance: 0.9, Temp: 347.5, MaxPop: 0
1.81% Completion
1.88% Completion
1.94% Completion
2.0% Completion
2.06% Completion
2.12% Completion
2.19% Completion
2.25% Completion
2.31% Completion

/mnt/c/Users/Ethanguest.DESKTOP-VRAD7JU/OneDrive/coupled_ebm/model/coreFunctions.py:273: RuntimeWarning: divide by zero encountered in double_scalars
  return gen20*(10/percentDeadByEnd)
/mnt/c/Users/Ethanguest.DESKTOP-VRAD7JU/OneDrive/coupled_ebm/model/coreFunctions.py:273: RuntimeWarning: invalid value encountered in double_scalars
  return gen20*(10/percentDeadByEnd)


Distance: 1.0, Temp: 352.6, MaxPop: 10
9.44% Completion
Distance: 1.0, Temp: 355.2, MaxPop: 10
9.5% Completion
Distance: 1.0, Temp: 357.8, MaxPop: 10
9.56% Completion
Distance: 1.0, Temp: 360.3, MaxPop: 10
9.62% Completion
Distance: 1.0, Temp: 362.9, MaxPop: 10
9.69% Completion
Distance: 1.0, Temp: 365.5, MaxPop: 10
9.75% Completion
Distance: 1.0, Temp: 368.0, MaxPop: 10
9.81% Completion
Distance: 1.0, Temp: 370.6, MaxPop: 10
9.88% Completion
Distance: 1.0, Temp: 373.1, MaxPop: 10
9.94% Completion

 Runtime: 24.31 minutes have passed
 Runtime: 0.41 hours have passed
 

 Distance:  0.956  AU
10.0% Completion
10.06% Completion
10.12% Completion
10.19% Completion
10.25% Completion
10.31% Completion
10.38% Completion
10.44% Completion
10.5% Completion
10.56% Completion
10.62% Completion
Distance: 1.0, Temp: 301.4, MaxPop: 0
10.69% Completion
Distance: 1.0, Temp: 303.9, MaxPop: 0
10.75% Completion
Distance: 1.0, Temp: 306.5, MaxPop: 0
10.81% Completion
Distance: 1.0, Temp: 309.0, MaxPop: 1


KeyboardInterrupt: 

## Plotting

In [ ]:
df = pd.read_csv('dataSweep_03_25_NMax_10_billion.csv')
filter = df["logpco20"] <= 5  # making boolean series 
for i in np.arange(3,18,1):
    df[df.columns[i]].where(filter, inplace = True) #filter data

# peakPopRatioList = np.log10((df["peakPopList"]/df["carryCapList"]))
# df['peakPopRatio'] = peakPopRatioList
def tCollapse(gamma):
    if(np.isnan(gamma)): return np.nan
    else:
        return max(1, 1/gamma)
df['normCollapseTime'] = df['gamma'].apply(tCollapse)
A0 = 0.04
df['logCollapseTime'] = np.log10(df['normCollapseTime']/A0)
# for i in np.asarray(df['collapseTime']):
#     print(i)
df6 = df[["dist", "temp", "logCollapseTime"]]
df6 = df6.pivot("dist", "temp", "logCollapseTime")
x4 = np.asarray(df6.index)
y4 = np.asarray(df6.columns)
xx4, yy4 = np.meshgrid(x4,y4)
zz4 = np.zeros_like(xx4)
for i in range(zz4.shape[0]):
    for j in range(zz4.shape[1]):
        zz4[j,i] = df6.iloc[i,j]


df5 = df[["dist", "temp", "logdeclineTime"]]
df5 = df5.pivot("dist", "temp", "logdeclineTime")
x3 = np.asarray(df5.index)
y3 = np.asarray(df5.columns)
xx3, yy3 = np.meshgrid(x3,y3)
zz3 = np.zeros_like(xx3)
for i in range(zz3.shape[0]):
    for j in range(zz3.shape[1]):
        zz3[j,i] = df5.iloc[i,j]


df4 = df[["dist", "temp", "peakPopList"]]
df4 = df4.pivot("dist", "temp", "peakPopList")
x2 = np.asarray(df4.index)
y2 = np.asarray(df4.columns)
xx2, yy2 = np.meshgrid(x2,y2)
zz2 = np.zeros_like(xx2)
for i in range(zz2.shape[0]):
    for j in range(zz2.shape[1]):
        zz2[j,i] = df4.iloc[i,j]
        
        
df2 = df[["dist", "temp", "logpco20"]]
df2 = df2.pivot("dist", "temp", "logpco20")
x = np.asarray(df2.index)
y = np.asarray(df2.columns)
xx, yy = np.meshgrid(x,y)
zz = np.zeros_like(xx)
for i in range(zz.shape[0]):
    for j in range(zz.shape[1]):
        zz[j, i] = df2.iloc[i,j]
            
df3 = df[["dist", "temp", "logGamma"]]
df3 = df3.pivot("dist", "temp", "logGamma")
x1 = np.asarray(df3.index)
y1 = np.asarray(df3.columns)
xx1, yy1 = np.meshgrid(x1,y1)
zz1 = np.zeros_like(xx1)
for i in range(zz1.shape[0]):
    for j in range(zz1.shape[1]):
        zz1[j,i] = df3.iloc[i,j]


fig, axes = plt.subplots(figsize=(25,30), nrows=3, ncols=2)
#-----------------------------------log-Pco20---------------------------------------------------------
contourLines = np.asarray([0,1,2,3,4, 4.5])
ax = axes[0,0]
colorMap = "plasma_r"
cmm = plt.get_cmap(colorMap)
cmm.set_bad(color='grey')
cs1 = ax.pcolormesh(yy, xx, zz, alpha=0.7,cmap=cmm)
#cs2 = ax.contour(yy, xx, zz, cmap=colorMap, levels=contourLines)
cs2 = ax.contour(yy, xx, zz, cmap=cmm, levels=contourLines)
#ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1.1f}\ ppm$") #set contour labels
ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $log\ pCO_{2}\ (ppm)$") #set axis labels;
ax.set_xlim(273.15,360)
cbar = fig.colorbar(cs1, ax=ax);
cbar.set_label(r"$log\ pCO_{2}\ (ppm)$")

#-----------------------------------log-Gamma---------------------------------------------------------
#contourLines = np.asarray([-4,-3,-2,-1,0])
ax = axes[0,1]
colorMap = "viridis"
cmm = plt.get_cmap(colorMap)
cmm.set_bad(color='grey')
cs1 = ax.pcolormesh(yy1, xx1, zz1, alpha=0.9,cmap=cmm)
#cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap, levels=contourLines)
cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap)
#ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1.1f}$") #set contour labels
ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $log\ Gamma\ (log\ \gamma)$") #set axis labels;
ax.set_xlim(273.15,360)
cbar = fig.colorbar(cs1, ax=ax);
cbar.set_label(r"$log\ Gamma\ (log\ \gamma)$")

# #-----------------------------------log-Gamma-pCo2------------------------------------------------------
# #contourLines = np.asarray([-4,-3,-2,-1,0])
contourLines = np.asarray([0,1,2,3,4, 4.5])
ax = axes[2,1]
colorMap = "viridis"
cmm = plt.get_cmap(colorMap)
cmm.set_bad(color='grey')
cs1 = ax.pcolormesh(yy1, xx1, zz1, alpha=0.9,cmap=cmm)
#cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap, levels=contourLines)
cs2 = ax.contour(yy, xx, zz, cmap="plasma_r", levels=contourLines)
#ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1.1f}\ ppm$") #set contour labels
ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $log\ Gamma$  (Contoured by $pCO2$)") #set axis labels;
ax.set_xlim(273.15,360)
cbar = fig.colorbar(cs1, ax=ax);
cbar.set_label(r"$log\ Gamma\ (log\ \gamma)$")

# #-----------------------------------tCollapse------------------------------------------------------
# #contourLines = np.asarray([-4,-3,-2,-1,0])
ax = axes[1,1]
colorMap = "jet"
cmm = plt.get_cmap(colorMap)
cmm.set_bad(color='grey')
cs1 = ax.pcolormesh(yy4, xx4, zz4, alpha=0.7,cmap=cmm)
#cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap, levels=contourLines)
cs2 = ax.contour(yy4, xx4, zz4, cmap=cmm)
#ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
ax.clabel(cs2, inline=True, fontsize=14, colors="Black",  fmt=r"$10^{%1.1f}\ yr$") #set contour labels
ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $log\ t_{collapse}\ (yrs)$)") #set axis labels;
ax.set_xlim(273.15,360)
cbar = fig.colorbar(cs1, ax=ax);
cbar.set_label(r"$log\ t_{collapse}\ (yrs)$")
#-----------------------------------Peak Population------------------------------------------------------
contourLines = np.asarray([10, 30, 50, 70,90])
ax = axes[2,0]
colorMap = "jet"
cmm = plt.get_cmap(colorMap)
cmm.set_bad(color='grey')
cs7 = ax.pcolormesh(yy2, xx2, zz2, alpha=0.7,cmap=cmm)
cs8 = ax.contour(yy2, xx2, zz2, cmap=cmm, levels=contourLines)
ax.clabel(cs8, inline=True, fontsize=14, colors="Black", fmt=r"${%1.0f}$") #set contour labels
#ax.clabel(cs8, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1.1f}\ ppm$) #set contour labels
#ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\$") #set contour labels
ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU) vs  Temperature  (K)  vs  $Peak\ Population\ (billion)$") #set axis labels;
ax.set_xlim(273.15,360)
cbar = fig.colorbar(cs7, ax=ax);
cbar.set_label(r"$Peak\ Population\ (billion)$")
#-----------------------------------log-tDecline------------------------------------------------------
ax = axes[1,0]
colorMap = "jet"
cmm = plt.get_cmap(colorMap)
cmm.set_bad(color='grey')
cs5 = ax.pcolormesh(yy3, xx3, zz3, alpha=0.7,cmap=cmm)
cs6 = ax.contour(yy3, xx3, zz3, cmap=cmm)
ax.clabel(cs6,  fontsize=14, colors="Black",  fmt=r"$10^{%1.1f}\ yr$") #set contour labels
#ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\$") #set contour labels
ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU) vs  Temperature  (K)  vs  $log\ t_{decline}\ (yrs)$") #set axis labels;
ax.set_xlim(273.15,360)
cbar = fig.colorbar(cs5, ax=ax);
cbar.set_label(r"$log\ t_{decline}\ (yrs)$")
plt.suptitle("Carrying Capacity = 10 Billion", y=.9,fontsize=20)
plt.savefig("informationalPlots_NMax_10_Billion.png")
plt.show()

In [ ]:
# df = pd.read_csv('dataSweep_3_21.csv')

# df5 = df[["dist", "temp", "logtYear"]]
# df5 = df5.pivot("dist", "temp", "logtYear")
# x3 = np.asarrdx vcay(df5.index)
# y3 = np.asarray(df5.columns)
# xx3, yy3 = np.meshgrid(x3,y3)
# zz3 = np.zeros_like(xx3)
# for i in range(zz3.shape[0]):
#     for j in range(zz3.shape[1]):
#         zz3[j,i] = df5.iloc[i,j]


# df4 = df[["dist", "temp", "logPopDeath"]]
# df4 = df4.pivot("dist", "temp", "logPopDeath")
# x2 = np.asarray(df4.index)
# y2 = np.asarray(df4.columns)
# xx2, yy2 = np.meshgrid(x2,y2)
# zz2 = np.zeros_like(xx2)
# for i in range(zz2.shape[0]):
#     for j in range(zz2.shape[1]):
#         zz2[j,i] = df4.iloc[i,j]
        
        
# df2 = df[["dist", "temp", "logpco20"]]
# df2 = df2.pivot("dist", "temp", "logpco20")
# x = np.asarray(df2.index)
# y = np.asarray(df2.columns)
# xx, yy = np.meshgrid(x,y)
# zz = np.zeros_like(xx)
# for i in range(zz.shape[0]):
#     for j in range(zz.shape[1]):
#         zz[j, i] = df2.iloc[i,j]
            
# df3 = df[["dist", "temp", "logGamma"]]
# df3 = df3.pivot("dist", "temp", "logGamma")
# x1 = np.asarray(df3.index)
# y1 = np.asarray(df3.columns)
# xx1, yy1 = np.meshgrid(x1,y1)
# zz1 = np.zeros_like(xx1)
# for i in range(zz1.shape[0]):
#     for j in range(zz1.shape[1]):
#         zz1[j,i] = df3.iloc[i,j]


# fig, axes = plt.subplots(figsize=(22,18), nrows=2, ncols=2)

# #-----------------------------------log-Gamma---------------------------------------------------------
# #contourLines = np.asarray([-4,-3,-2,-1,0])
# ax = axes[0,0]
# colorMap = "plasma_r"
# cmm = plt.get_cmap(colorMap)
# cmm.set_bad(color='grey')
# cs1 = ax.pcolormesh(yy, xx, zz, alpha=0.7,cmap=cmm)
# #cs2 = ax.contour(yy, xx, zz, cmap=colorMap, levels=contourLines)
# cs2 = ax.contour(yy, xx, zz, cmap=cmm)
# #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
# ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\ ppm$") #set contour labels
# ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $log\ pCO_{2}\ (ppm)$") #set axis labels;
# cbar = fig.colorbar(cs1, ax=ax);
# cbar.set_label(r"$log\ pCO_{2}\ (ppm)$")

# #-----------------------------------log-Gamma---------------------------------------------------------
# #contourLines = np.asarray([-4,-3,-2,-1,0])
# ax = axes[0,1]
# colorMap = "viridis"
# cmm = plt.get_cmap(colorMap)
# cmm.set_bad(color='grey')
# cs1 = ax.pcolormesh(yy1, xx1, zz1, alpha=0.7,cmap=cmm)
# #cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap, levels=contourLines)
# cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap)
# #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
# ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}$") #set contour labels
# ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $log\ Gamma\ (log\ \gamma)$") #set axis labels;
# cbar = fig.colorbar(cs1, ax=ax);
# cbar.set_label(r"$log\ Gamma\ (log\ \gamma)$")

# # #-----------------------------------log-Gamma-pCo2------------------------------------------------------
# # #contourLines = np.asarray([-4,-3,-2,-1,0])
# # ax = axes[1,0]
# # colorMap = "viridis"
# # cmm = plt.get_cmap(colorMap)
# # cmm.set_bad(color='grey')
# # cs1 = ax.pcolormesh(yy1, xx1, zz1, alpha=0.7,cmap=cmm)
# # #cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap, levels=contourLines)
# # cs2 = ax.contour(yy, xx, zz, cmap="plasma_r")
# # #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
# # ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\ ppm$") #set contour labels
# # ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $log\ Gamma$  (Contoured by $pCO2$)") #set axis labels;
# # cbar = fig.colorbar(cs1, ax=ax);
# # cbar.set_label(r"$log\ Gamma\ (log\ \gamma)$")

# #-----------------------------------log-Popdeath------------------------------------------------------
# #contourLines = np.asarray([-4,-3,-2,-1,0])
# ax = axes[1,1]
# colorMap = "gist_heat"
# cmm = plt.get_cmap(colorMap)
# cmm.set_bad(color='grey')
# cs7 = ax.pcolormesh(yy2, xx2, zz2, alpha=0.7,cmap=cmm)
# cs8 = ax.contour(yy2, xx2, zz2, cmap=cmm)
# ax.clabel(cs8, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\ ppl$") #set contour labels
# #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\$") #set contour labels
# ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU) vs  Temperature  (K)  vs  $log\ PopDeaths$") #set axis labels;
# cbar = fig.colorbar(cs7, ax=ax);
# cbar.set_label(r"$log\ PopDeaths$")
# #-----------------------------------log-tDeath------------------------------------------------------
# #contourLines = np.asarray([-4,-3,-2,-1,0])
# ax = axes[1,0]
# colorMap = "gist_heat_r"
# cmm = plt.get_cmap(colorMap)
# cmm.set_bad(color='grey')
# cs5 = ax.pcolormesh(yy3, xx3, zz3, alpha=0.7,cmap=cmm)
# cs6 = ax.contour(yy3, xx3, zz3, cmap=cmm)
# ax.clabel(cs6, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\ yr$") #set contour labels
# #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}\$") #set contour labels
# ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU) vs  Temperature  (K)  vs  $log\ t_{ff}$") #set axis labels;
# cbar = fig.colorbar(cs5, ax=ax);
# cbar.set_label(r"$log\ t_{ff}$")

In [ ]:
#fig = plt.figure(figsize=(17,7))
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("dataSweep_3_21.csv")
# df2 = df[df.logGamma.isna() & df["dist"]<1.0]
# #print(print(df2[df2['temp'].notna()]))
# ind=np.asarray(df.index.difference(df2.index))
# for i in in d:
#     print(df.iloc[i])
# df["dist"] = np.round(distanceList,3) #x
# df["temp"] = np.round(tempList)     #y
# df["logdTdP"] = np.round(np.log10(dTdPList))
# df["logpco20"] = np.log10(pco2List)
# df["gamma"] = dimVarList
# df["logGamma"] = np.log10(dimVarList)
# df["tFreeFall"] = -np.log10(dimVarList)

# A=0.04;#birth ratee
# tFreeFall = lambda g: min(1, 1/g)
# df["tFreeFall"] = df["gamma"].apply(tFreeFall)
# df["tGen"] = df["tFreeFall"]/A
#print(df[df["logpco20"].isna()])

df2 = df[["dist", "temp", "logpco20"]]
df2 = df2.pivot("dist", "temp", "logpco20")

colorMap = "plasma_r"
x = np.asarray(df2.index)
y = np.asarray(df2.columns)
xx, yy = np.meshgrid(x,y)
zz = np.zeros_like(xx)
for i in range(zz.shape[0]):
    for j in range(zz.shape[1]):
        zz[j, i] = df2.iloc[i,j]
            
df3 = df[["dist", "temp", "logGamma"]]
df3 = df3.pivot("dist", "temp", "logGamma")
colorMap = "viridis"
x1 = np.asarray(df3.index)
y1 = np.asarray(df3.columns)
xx1, yy1 = np.meshgrid(x1,y1)
zz1 = np.zeros_like(xx1)
for i in range(zz1.shape[0]):
    for j in range(zz1.shape[1]):
        zz1[j,i] = df3.iloc[i,j]

contourLines = np.asarray([-4,-3,-2,-1,0])
fig, ax = plt.subplots(figsize=(10,7.5))
fig.subplots_adjust(top=.8)
cs1 = ax.pcolormesh(yy1, xx1, zz1, alpha=0.7,cmap=colorMap)
cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap, levels=contourLines)
#palette = plt.cm.jet
#palette.set_bad ('w',0.3) # Bad values (i.e., masked, set to grey 0.8 
#ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}$") #set contour labels
ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $logGamma\ (log\ \gamma)$") #set axis labels;
cbar = fig.colorbar(cs1, ax=ax);
cbar.set_label(r"$logGamma\ (log\ \gamma)$")
#plt.savefig("summaryFig2.png")

In [ ]:
#fig = plt.figure(figsize=(17,7))
# import warnings
# warnings.filterwarnings("ignore")

# df = pd.read_csv("prodData.csv")

# A=0.04;#birth ratee
# tFreeFall = lambda g: max(1, 1/g)
# df["tFreeFall"] = df["gamma"].apply(tFreeFall)
# df["tGen"] = df["tFreeFall"]/A
# #print(df[df["logpco20"].isna()])

# df2 = df[["logGamma", "logdTdP", "tGen"]]
# # df2 = df2.drop_duplicates().reset_index()
# # print(df2)
# df2 = df2.pivot("logGamma", "logdTdP", "tGen")
# colorMap = "plasma_r"
# x = np.asarray(df2.index)
# y = np.asarray(df2.columns)
# xx, yy = np.meshgrid(x,y)
# zz = np.zeros_like(xx)
# for i in range(20):
#     for j in range(zz.shape[1]-2):
#         zz[j, i] = df2.iloc[i,j]
            

# # contourLines = np.asarray([-4,-3,-2,-1,0])
# fig, ax = plt.subplots(figsize=(10,7.5))
# fig.subplots_adjust(top=.8)
# cs1 = ax.pcolormesh(yy, xx, zz, alpha=0.7,cmap=colorMap)
# cs2 = ax.contour(yy, xx, zz, cmap=colorMap, levels=contourLines)
# #palette = plt.cm.jet
# #palette.set_bad ('w',0.3) # Bad values (i.e., masked, set to grey 0.8 
# #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
# ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}$") #set contour labels
# ax.set(xlabel=r'$dTdP\ (K/ppm)$', ylabel=r'$Gamma\ (\gamma)$', title=r"dTdP  (K/ppm)  vs  Gamma  ($\gamma$)  vs  Collapse Timescale $(t_{years})$") #set axis labels;
# cbar = fig.colorbar(cs1, ax=ax);
# cbar.set_label(r"$logGamma\ (log\ \gamma)$")
#plt.savefig("summaryFig3.png")

In [ ]:
# df = pd.DataFrame()
# df["dist"] = np.round(distanceList,3) #x
# df["temp"] = np.round(tempList)     #y
# df["dTdP"] = np.round(np.log10(dTdPList))
# df["pco20"] = np.log10(pco2List)
# df["gamma"] = np.log10(dimVarList)

# fig, axes = plt.subplots(figsize=(15,5), nrows=1, ncols=2)
# cmap_ = "YlGnBu"

# ax = axes[0]
# df2 = df.pivot("dist", "temp", "gamma")
# sns.heatmap(df2, ax=ax, cmap = "Greens",  annot=True)
# ax.set_ylabel("Distance (AU)")
# ax.set_xlabel("Temperature (K)")
# ax.set_title("Distance vs Temperature vs LogGamma")

# ax = axes[1]
# df3 = df.pivot("dist", "temp", "pco20")
# sns.heatmap(df3, ax=ax, cmap = "Purples", linewidths=.2)
# ax.set_ylabel("Distance (AU)")
# ax.set_xlabel("Temperature (K)")
# ax.set_title("Distance vs Temperature vs LogpCO20")


# plt.tight_layout()
# plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(15,5))
contourLines = np.linspace(-90, -15, 4)

dfNew = pd.DataFrame()
x, y = np.linspace(-10,5,5), np.linspace(-10,10,5)
z = lambda x, y: y**2

xList = []
yList = []
zList = []
for i in x:
    for j in y:
        xList.append(i)
        yList.append(j)
        zList.append(z(i,j))
dfNew["x"] = xList
dfNew["y"] = yList
dfNew["z"] = zList
dfNew = dfNew.pivot("x", "y", "z")

#ax = fig.add_subplot(1 , 3, 1)
ax = plt.gca()
sns.heatmap(dfNew, ax=ax,annot=True)
plt.show()
#ax.set(xticks=[], yticks=[])

xx, yy = np.meshgrid(x,y)
zz = np.zeros_like(xx)
for i in range(zz.shape[0]):
    for j in range(zz.shape[1]):
        zz[j,i] = dfNew.iloc[i,j]

# ax = fig.add_subplot(1 , 3, 2)
# cs = ax.contour(xx, yy, zz, levels=contourLines, cmap='coolwarm')
# ax.clabel(cs, inline=True, fontsize=12)
# ax.set(xlabel=r'$X$', ylabel=r'$Y$')
# fig.tight_layout()


# ax = fig.add_subplot(1 , 3, 3, projection='3d')
# ax.plot_surface(xx, yy, zz, cmap='coolwarm', alpha=0.5)
# for level, contour in zip(contourLines, cs.allsegs):
#     dat0 = contour[0]
#     ax.plot(dat0[:, 0], dat0[:, 1], level, color='k')
#     ax.text(dat0[0, 0], dat0[0, 1], level, '{0}'.format(level), zdir=(1, 0, 0), ha='center', va='top')
# ax.view_init(elev=20, azim=-45)
# ax.set(xlabel=r'$X$', ylabel=r'$Y$', zlabel=r'$Z$', title=r'$Z = -(X^2 + Y^2)$')
# plt.tight_layout()
# plt.show()
fig = plt.figure()
ax = plt.gca()
cs = plt.contourf(yy, xx, zz, alpha=.5)
cs2 = plt.contour(yy, xx, zz)
ax.clabel(cs2)
fig.colorbar(cs)
plt.show()
# plt.pcolormesh(xx,yy,zz)